In [ ]:
# Get hidden states by context level and by layer

In [1]:
import os
os.environ['HF_HOME'] = '/sbksvol/amurali/'

from huggingface_hub import login
login(token = "<hf_token>")

import transformers
import torch

import csv
import numpy as np
import pandas as pd

from transformers import AutoConfig, AutoModel, AutoTokenizer, AutoModelForCausalLM, pipeline, LlamaForCausalLM

from tqdm import tqdm

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /sbksvol/amurali/token
Login successful


In [2]:
print(f"PyTorch Version: {torch.__version__}")
print(f"Transformers Version: {transformers.__version__}")
print(f"NumPy Version: {np.__version__}")

PyTorch Version: 1.13.1+cu117
Transformers Version: 4.30.2
NumPy Version: 1.21.2


In [3]:
# Load model/create pipeline
model_name = "ivnle/llamatales_jr_8b-lay8-hs512-hd8-33M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, output_hidden_states=True).to("cuda")
llamatales_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

# Prompts
prompts = {
    1: "Once upon a time there was a dragon", 
    2: "Once upon a time there was a princess", 
    3: "Once upon a time there were two children",
    4: "Once upon a time there was a prince",
    5: "Once upon a time there was a frog",
    6: "Once upon a time there was a king",
    7: "Once upon a time there was a queen",
    8: "Once upon a time there was a wolf",
    9: "Once upon a time there was a genie",
    10: "Once upon a time there was a poor boy"
}


for prompt_id, prompt_text in prompts.items():
    print(f"Prompt {prompt_id}: \"{prompt_text}\"")
    data = []
    npz_data = {}
    hidden_state_file = f'./hidden_states/prompt_{prompt_id}.npz'
    for i in tqdm(range(1000)):
        # Generate using pipeline
        sequences = llamatales_pipeline(
            prompt_text,
            do_sample=True,
            top_k=10,
            num_return_sequences=1,
            max_new_tokens=100
        )
        generated_story = sequences[0]['generated_text']
#         print(f"\nGenerated (pipeline):\n{generated_story}")

        # Count tokens
        num_tokens_generated_story = len(tokenizer.encode(generated_story))

        # Generate hidden states from model.generate using same prompt
        inputs = tokenizer(generated_story, return_tensors="pt").to("cuda")
        outputs = model.generate(
            inputs.input_ids,
            attention_mask=inputs.attention_mask,
            do_sample=True,
            top_k=10,
            num_return_sequences=1,
            max_new_tokens=1,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
            return_dict_in_generate=True,
            output_hidden_states=True
        )
        
        new_story = tokenizer.batch_decode(outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)[0]
#         print("New Story: \n", new_story)
        
#         print("Length of Generated Story: ", num_tokens_generated_story)
#         print("Length of New Story: ", len(tokenizer.encode(new_story)))
        
        # Map output tokens to prompt ID
        output_id = [prompt_id] * num_tokens_generated_story

        # Extract hidden states
        convert_hidden_states = []
#         print(f"Type of hidden_states: {type(outputs.hidden_states)}")
        for layer_hidden_states in outputs.hidden_states:
            convert_hidden_states.append([t.detach().cpu().numpy() for t in layer_hidden_states])

        # Debug hidden state shape
#         print(f"Hidden states shape: {len(convert_hidden_states)} generated tokens × "
#               f"{len(convert_hidden_states[0])} layers × "
#               f"{len(convert_hidden_states[0][0])} sequences × "
#               f"{len(convert_hidden_states[0][0][0])} tokens × " 
#               f"{len(convert_hidden_states[0][0][0][0])} dims")

#         print("Num Tokens: ", len(convert_hidden_states[0][0][0])) #same as context (length of original generated story)

        # Save hidden states to file
        npz_data[f"arr_{i}"] = convert_hidden_states
        
        # Store results
        data.append([prompt_id, prompt_text, generated_story, hidden_state_file, num_tokens_generated_story, len(tokenizer.encode(new_story))])
    
    np.savez_compressed(hidden_state_file, **npz_data)
    
    df = pd.DataFrame(data, columns=["prompt_id", "prompt", "story", "hidden_state_file", "len_generated_story", "len_new_story"])
    if(prompt_id == 1): df.to_csv("story_dataset_new.csv", index = False)
    else:
        df.to_csv('story_dataset_new.csv', mode='a', header = False, index = False)
        


Some weights of LlamaForCausalLM were not initialized from the model checkpoint at ivnle/llamatales_jr_8b-lay8-hs512-hd8-33M and are newly initialized: ['model.layers.0.self_attn.rotary_emb.inv_freq', 'model.layers.1.self_attn.rotary_emb.inv_freq', 'model.layers.6.self_attn.rotary_emb.inv_freq', 'model.layers.3.self_attn.rotary_emb.inv_freq', 'model.layers.2.self_attn.rotary_emb.inv_freq', 'model.layers.4.self_attn.rotary_emb.inv_freq', 'model.layers.7.self_attn.rotary_emb.inv_freq', 'model.layers.5.self_attn.rotary_emb.inv_freq']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


Prompt 1: "Once upon a time there was a dragon"


  0%|                                                                                                         | 0/1000 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/transformers/generation/utils.py:1260: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  "You have modified the pretrained model configuration to control generation. This is a"
  1%|▉                                                                                               | 10/1000 [00:17<26:41,  1.62s/it]/opt/conda/lib/python3.7/site-packages/transformers/pipelines/base.py:1084: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,
100%|█████████████████████████████████████

Prompt 2: "Once upon a time there was a princess"


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [27:49<00:00,  1.67s/it]


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [30:57<00:00,  1.86s/it]


Prompt 8: "Once upon a time there was a wolf"


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [29:29<00:00,  1.77s/it]


Prompt 9: "Once upon a time there was a genie"


 28%|██████████████████████████▏                                                                    | 275/1000 [08:31<18:14,  1.51s/it]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [33:47<00:00,  2.03s/it]


In [4]:
df = pd.read_csv("story_dataset_new.csv")
df

,prompt_id,prompt,story,hidden_state_file,len_generated_story,len_new_story
0,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Blaz...,./hidden_states/prompt_1.npz,270,271
1,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Spar...,./hidden_states/prompt_1.npz,349,350
2,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Scor...,./hidden_states/prompt_1.npz,278,278
3,1,Once upon a time there was a dragon,Once upon a time there was a dragon. The drago...,./hidden_states/prompt_1.npz,117,118
4,1,Once upon a time there was a dragon,Once upon a time there was a dragon. The drago...,./hidden_states/prompt_1.npz,129,130
...,...,...,...,...,...,...
9995,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,289,290
9996,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,119,119
9997,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,127,128
9998,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,441,441
